In [4]:
import os
import re
import shutil
import src.utils as utils
import src.Parser.verilog_parser as verilog_parser
import src.Parser.bench_parser as bench_parser
import src.Parser.conv as conv
from  src.Netlist.AST import AST,module
import src.Attacks.SATAttack.SATAttack as satattack
import src.Attacks.SATAttack.benchmarks_custom as benchmarks

In [5]:
# top="test_adder"
# netlist = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format

In [ ]:
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/65GP_STDCELL_verilog/tcbn65gplus.v"
# path= "vlib/mycells.v"
# path="input_files/warm-up_obfuscation/internal_lib.v"
path="/home/alira/FYP_FINAL/input_files/ASSURE_LOCKED/modulefiles.v"
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules/Train_add_mul_sub_64_bit_Syn_65nm.v"
with open(path,"r") as f:
  module_txt=f.read()
gate_mapping_vlib,gates_vlib,FF_vlib=verilog_parser.extract_modules_def(module_txt)
vlib_var=gate_mapping_vlib,gates_vlib,FF_vlib
bench=open("input_files/ASSURE_LOCKED/design4/design4.bench").read()

In [ ]:
clkpin="Clock"
modulename="top"
gate_mapping_vlib,gates_vlib,FF_vlib=vlib_var
bench=re.sub(" +"," ",bench)
gates,gate_count = bench_parser.extract_gates_b(bench)
inputs = bench_parser.extract_io_b(bench, mode="input")
outputs = bench_parser.extract_io_b(bench, mode="output")

In [ ]:
gate_list = list(gates.keys())
gate_list.remove("DFF")
gate_list

In [ ]:
if(("DFF" in gate_list) and clkpin not in inputs):
    print("DFF IN CIRCUIT")
    clockp=","+clkpin
    clockio="input {};".format(clkpin)
else:
    clockio=""
    clockp=""

porti, input_dec,replace_i = utils.io_port(inputs, mode="input")
porto, output_dec,replace_o = utils.io_port(outputs, mode="output")

In [ ]:
verilog = "module {} ({},{});{}".format(modulename,
    porti+clockp, porto, input_dec+output_dec+clockio)

In [ ]:
def gen_line(i,j,inputs,output,port,module_name):
    tmp={}
    for mIO,mNode in zip(inputs,j[1:]):
        tmp[mIO]=mNode
    tmp[output]=j[0]

    formatted_out=re.sub(r"[^A-Za-z0-9_]",r"",j[0])
    init_name=f"{i}_out_{formatted_out}_"
    return f"{module_name} {init_name} {port.format(**tmp)}\n"


def proc_nested(i,out,inp,inputs,output,port,module_name,layer=0):
    tmpouts=[]
    txt=""
    for c,t in enumerate(range(0,len(inp)-1,2)):
        tmpi=f"{out}_{layer}_{c}"
        tmpouts.append(tmpi)
        txt+=gen_line(i,[tmpi]+inp[t:t+2],inputs,output,port,module_name)

    if(inp[t+2:]):
        tmpouts=tmpouts+inp[t+2:]

    if(len(tmpouts)==2):
        txt+=gen_line(i,[out]+tmpouts,inputs,output,port,module_name)
        return tmpouts,txt
    
    tmpouts_r,txt_r=proc_nested(i,out,tmpouts,inputs,output,port,module_name,layer+1)
    return tmpouts+tmpouts_r,txt+txt_r

def gen_line_multi(i,j,inputs,output,port,module_name):
    return proc_nested(i,j[0],j[1:],inputs,output,port,module_name,layer=0)


In [ ]:
i="DFF"
Node = gates[i]
for k in FF_vlib:
    inputs=FF_vlib[k]["inputs"]
    port=FF_vlib[k]["port"]
    if(len(inputs)==2):
        template=FF_vlib[k]
        break

port=template["port"]
inputs=template["inputs"]
output=template["outputs"][0]
module_name=k

In [ ]:
wires=[]
verilog_wire=""
verilog_init=""

In [ ]:
import concurrent.futures
from multiprocessing import Manager


def test(Nodeseg):
    print("H")
    local_wires=[]
    local_verilog_init=""
    # local_verilog_wire=""
    for j in Nodeseg:
        if((j[0] not in outputs) and (j[0] not in local_wires)):
            # local_verilog_wire +="wire {};".format(j[0])
            local_wires.append(j[0])
    
        tmp={}
        for mIO in template["port_list"]:
            type_node=utils.det_FF_node(mIO)
            if(type_node=="clock"):
                tmp[mIO]=clkpin
            elif(type_node=="inputs"):
                tmp[mIO]=j[1]
            elif(type_node=="outputs"):
                tmp[mIO]=j[0]
        init_name=f"DFF_out_{j[0]}_"
        # print(f"{module_name} {init_name} {port.format(**tmp)}\n")
        local_verilog_init+=f"{module_name} {init_name} {port.format(**tmp)}\n"
        
    return local_wires,local_verilog_init
    # shared_wires.value+=local_wires
    # shared_verilog_init.value+=local_verilog_init
    # shared_verilog_wire.value+=local_verilog_wire


def process_FF_node(Nodeseg,shared_wires,shared_verilog_init,shared_verilog_wire):
    local_wires=[]
    local_verilog_init=""
    local_verilog_wire=""
    for j in Nodeseg:
        if((j[0] not in outputs) and (j[0] not in shared_wires.value) and (j[0] not in local_wires)):
            local_verilog_wire +="wire {};".format(j[0])
            local_wires.append(j[0])
    
        tmp={}
        for mIO in template["port_list"]:
            type_node=utils.det_FF_node(mIO)
            if(type_node=="clock"):
                tmp[mIO]=clkpin
            elif(type_node=="inputs"):
                tmp[mIO]=j[1]
            elif(type_node=="outputs"):
                tmp[mIO]=j[0]
        init_name=f"DFF_out_{j[0]}_"
        # print(f"{module_name} {init_name} {port.format(**tmp)}\n")
        local_verilog_init+=f"{module_name} {init_name} {port.format(**tmp)}\n"
    shared_wires.value+=local_wires
    shared_verilog_init.value+=local_verilog_init
    shared_verilog_wire.value+=local_verilog_wire


def process_FF_node_iter(Nodeseg):
    global verilog_init,wires,verilog_wire
    for j in Nodeseg:
        if((j[0] not in outputs) and (j[0] not in wires)):
            verilog_wire +="wire {};".format(j[0])
            wires.append(j[0])
    
        tmp={}
        for mIO in template["port_list"]:
            type_node=utils.det_FF_node(mIO)
            if(type_node=="clock"):
                tmp[mIO]=clkpin
            elif(type_node=="inputs"):
                tmp[mIO]=j[1]
            elif(type_node=="outputs"):
                tmp[mIO]=j[0]
        init_name=f"DFF_out_{j[0]}_"
        verilog_init+=f"{module_name} {init_name} {port.format(**tmp)}\n"

In [ ]:
wires=[]
verilog_wire=""
verilog_init=""

In [ ]:


# Number of threads to use for parallel processing
num_threads = 4  # You can adjust this number based on your system's capabilities

# Split 'Node' list into chunks for parallel processing
chunk_size = len(Node) // num_threads
chunks = [Node[i:i+chunk_size] for i in range(0, len(Node), chunk_size)]


In [ ]:

def tmp1():
  manager = Manager()
  shared_wires = manager.Value(list,[])
  shared_verilog_init=manager.Value(str, "")
  shared_verilog_wire=manager.Value(str, "")

  with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as executor:
      executor.map(process_FF_node, chunks,[shared_wires]*num_threads,[shared_verilog_init]*num_threads,[shared_verilog_wire]*num_threads)

def tmp2():
  wires=[]
  verilog_wire=""
  verilog_init=""
  process_FF_node_iter(Node)

t=None
def test_in():
  global t
  with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as executor:
    t=executor.map(test, chunks)

In [ ]:
with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as executor:
  val=executor.map(test, chunks)

In [ ]:
i_wires=[]
i_txt=""
for i,j in val:
  i_txt+=j
  i_wires+=i

In [ ]:
i_wires.index('rst1')

In [ ]:
i_wires[0]

In [ ]:
for ic,i in enumerate(i_wires):
  if(i=='rst1'):
    print(ic)

In [ ]:
len(i_wires)

In [ ]:
# print(final_verilog_init)
with open("tmp.v","w") as f:
    f.write(i_txt)

In [ ]:
import timeit

t1=timeit.timeit(tmp1,number=1)
print("t1")
t2=timeit.timeit(tmp2,number=1)
print("t2")
t3=timeit.timeit(test_in,number=1)
print("t3")

In [ ]:
t3=timeit.timeit(test_in,number=1)
print("t3")

In [ ]:
for i in t:
    print(len(i))

In [ ]:
print(t1,t2,t3)

In [ ]:
def proc_node(Node,shared_wires,shared_verilog_init,shared_verilog_wire):
    local_wires=[]
    local_verilog_init=""
    local_verilog_wire=""
    for j in Node:
        if((j[0] not in outputs) and (j[0] not in shared_wires.value) and (j[0] not in local_wires)):
            local_verilog_wire +="wire {};\n".format(j[0])
            local_wires.append(j[0])
        if(len(j[1:])>2):
            new_wires_all,verilog_init_i=gen_line_multi(i,j,inputs,output,port,module_name)
            local_verilog_init+=verilog_init_i
            
            for tmpwire in new_wires_all:
                if((tmpwire not in outputs) and (tmpwire not in shared_wires.value) and (tmpwire not in local_verilog_wire)):
                    local_verilog_wire.append(tmpwire)
                    local_verilog_wire +="wire {};\n".format(tmpwire)
        else:
            local_verilog_init+=gen_line(i,j,inputs,output,port,module_name)
    shared_wires.value+=local_wires
    shared_verilog_init.value+=local_verilog_init
    shared_verilog_wire.value+=local_verilog_wire

In [ ]:
for i in gate_list:
    print(i)
    Node = gates[i]
    
    template=gates_vlib[gate_mapping_vlib[i][0]]
    port=template["port"]
    inputs=template["inputs"]
    output=template["outputs"][0]
    module_name=gate_mapping_vlib[i][0]
    
    # Number of threads to use for parallel processing
    num_threads = 4  # You can adjust this number based on your system's capabilities

    # Split 'Node' list into chunks for parallel processing
    chunk_size = len(Node) // num_threads
    chunks = [Node[i:i+chunk_size] for i in range(0, len(Node), chunk_size)]


    # with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as executor:
    #     # Process each chunk of nodes in parallel
    #     executor.map(proc_node, chunks)
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as executor:
        executor.map(proc_node, chunks,[shared_wires]*num_threads,[shared_verilog_init]*num_threads,[shared_verilog_wire]*num_threads)
              
verilog += verilog_wire + verilog_init + "endmodule"
    # break
# return verilog, gate_count

In [ ]:
# print(final_verilog_init)
with open("tmp.v","w") as f:
    f.write(verilog_init)

In [ ]:
# {i:len(gates[i]) for i in gates}
# gates["DFF"]

In [ ]:
with open("tmp.v","w") as f:
    f.write(verilog_init)

In [ ]:
# locked_filename="input_files/benchmark_bench/rnd/apex2_enc05.bench"
# unlocked_filename="input_files/benchmark_bench/original/apex2.bench"
# ASSURE_BENCH
# top="apex2_enc05"
# top="apex2"


top="fsm_0_obf"
locked = AST(file_path=f"output_files/ASSURE_BENCH/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format
top="fsm"
unlocked = AST(file_path=f"output_files/ASSURE_BENCH/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format



In [ ]:
tmp=[(i,j["inputs"],j["outputs"]) for i,j in locked.top_module.FF_tech['DFFQ_X1'].items()]

In [ ]:
locked.top_module.FF_tech

In [ ]:
for init,D,Q in tmp:
  print(init,D,Q)
  print(locked.top_module.FF_tech['DFFQ_X1'].pop(init))

In [ ]:
# locked.top_module.gates["BUF"]

In [ ]:
for init,D,Q in tmp:
  # print(init,D,Q)
  if(D in locked.top_module.io["inputs"]):
    print("I",init)
  elif(Q in locked.top_module.io["outputs"]):
    print("O",init)
  elif(D in locked.top_module.io["outputs"]):
    print("D in O",init)
  # else:
    # print("Inter",init,Q in locked.top_module.io["inputs"],D in locked.top_module.io["outputs"])
  # print(init,{'inputs': [D], 'outputs': Q})

In [ ]:
locked.top_module.gen_graph()

In [ ]:
import src.Attacks.SATAttack.benchmarks_custom as benchmarks
import src.Netlist.netlist as netlist

In [ ]:
analyze=netlist.Analyze_Netlist(locked.top_module,unlocked.top_module)

In [ ]:
for i,j in zip(locked.top_module.io['outputs'],unlocked.top_module.io['outputs']):
    print(i,j)

In [ ]:
keys={}
for ii in analyze.attack_outputs:
  print("START SAT ATTACK")
  satobj=analyze.attack_outputs[ii]
  satattackobj=satattack.SatAttack(file_type="obj",satobj=satobj)
  # satobj=satattack.SatAttack(file_type="b",locked_filename=locked_filename, unlocked_filename=unlocked_filename)
  print(satattackobj.run())
  print(satattackobj.iterations)
  keys[ii]=satattackobj.key
  # break

In [ ]:
final_key={}
for o,k in keys.items():
  for i in k:
    # print(i)
    if(i not in final_key):
      final_key[i]=keys[o][i]
      # print(i,keys[o][i])
    else:
      print(i,keys[o][i],final_key[i])

In [ ]:
final_key

In [ ]:
locked.top_module.io["outputs"],unlocked.top_module.io["outputs"]

In [ ]:
satattackobj.key

In [ ]:
satattackobj._key_string(satattackobj.key)

In [ ]:
satattackobj.key

In [ ]:
# import pickle
# pickle.dump(std1, file)

In [ ]:
top="fsm_0_obf"
pathin="input_files/ASSURE_LOCKED/design1/design1_netlist.v"

# top="fsm"
# pathin="input_files/ASSURE_LOCKED/design1/oracle1_netlist.v"

# utils.clean_dir("./tmp")

# obj=AST(file_path=pathin,rw="w",flag="v",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design

locked = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format

In [ ]:
# gate_mapping_vlib,gates_vlib,FF_vlib=locked.gate_mapping_vlib,locked.gates_vlib,locked.FF_vlib

In [ ]:
# locked.top_module.gates.keys(),locked.top_module.FF_tech.keys()

In [ ]:
# tech_map={i:i for i in list(locked.top_module.gates,locked.top_module.FF_tech)}

In [ ]:
# for gate, gate_list in gate_mapping_vlib.items():
#   print(gate, gate_list)

In [ ]:
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/65GP_STDCELL_verilog/tcbn65gplus.v"
# path= "vlib/mycells.v"
# path="input_files/warm-up_obfuscation/internal_lib.v"
path="/home/alira/FYP_FINAL/input_files/ASSURE_LOCKED/modulefiles.v"
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules/Train_add_mul_sub_64_bit_Syn_65nm.v"
with open(path,"r") as f:
  module_txt=f.read()
gate_mapping_vlib2,gates_vlib2,FF_vlib2=verilog_parser.extract_modules_def(module_txt)

In [ ]:
gate_mapping_vlib,gates_vlib,FF_vlib=locked.gate_mapping_vlib,locked.gates_vlib,locked.FF_vlib

In [ ]:
tech_map=verilog_parser.gen_tech_map(gate_mapping_vlib,FF_vlib,gates_vlib,gate_mapping_vlib2,FF_vlib2,gates_vlib2)

In [ ]:
txt=verilog_parser.gates_to_txt_techmap(locked.top_module.gates,gates_vlib2,tech_map)
txt+=verilog_parser.FF_to_txt_techmap(locked.top_module.FF_tech,FF_vlib2,tech_map)

In [ ]:
with open("tmp.v","w") as f:
    f.write(txt)

In [ ]:
# path_dir=r"/mnt/d/alis_files/LAPTOP/alis_files/university_files/GDRIVE/Hardware security FYP/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules"
# paths=[os.path.join(path_dir,i) for i in os.listdir(path_dir) if(i[:5]=="Train")]

# import re
# def analyze_init(paths):
#   items={}
#   for path in paths:
#     with open(path,"r") as f:
#       module_txt=f.read()
    
#     for i in re.findall(r"(\w+) (\w+) \((.*)\);",module_txt)[1:]:
#       tmpi=re.findall(r'\.(\S+)\( ?[^\(\),]+\)',i[2])
#       port="("
#       for k in tmpi[:-1]:
#         port+=f".{k}({{{k}}}), "
#       port+=f".{tmpi[-1]}({{{tmpi[-1]}}})"+");"
#       if(i[0] not in items):
#         items[i[0]]=[port]
#       elif(items[i[0]]!=[port]):
#         raise Exception("Items have mismatching initiation")
#   return items

In [ ]:
# items=analyze_init(paths)

In [ ]:

# with open("input_files/ASSURE_LOCKED/modulefiles.v","r") as f:
#   module_txt=f.read()


# with open("tmp/tmpmod.v","w") as f:
#   f.write(module_txt)


# # with open("tmp.v","w") as f:
# #   f.write(t)

# gate_mapping,gates,FF=verilog_parser.extract_modules_def(module_txt)

# path="input_files/ASSURE_LOCKED/design4/design4_netlist.v"
# # "input_files/test.v"
# # "input_files/ASSURE_LOCKED/design4/design4_netlist.v"




# # # verilog=tmp1()
# # verilog=tmp2()
# # with open("tmp/tmp.v","w") as f:
# #   f.write(verilog)
# # # print(verilog)

# # print("HERE")


In [ ]:
# gate_tech,sub_module,(FF_tech,Clock_pins,Reset_pins)=verilog_parser.gates_module_extraction(verilog,gate_mapping,gates,FF)

In [ ]:
# json_files=["./output_files/"+i for i in os.listdir("./output_files/") if os.path.isfile("./output_files/"+i)]
# orgfile_names=[i.split(".bench")[0] for i in os.listdir("input_files/benchmark_bench/original")]
# json_files.sort()

# return_top =lambda i:re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(open(i).read()), re.DOTALL)[0][1]

# orgfile_names=[[return_top(f"input_files/ASSURE_LOCKED/{i}/{k}") for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if("oracle" in k and ".v" in k)][0] for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]

# for i in json_files:
#   for j in orgfile_names:
#     if(j in i.split("/")[-1].split("org")[0]):
#       print(i,"output_files/ASSURE_VERILOG/"+i.split("/")[-1])
#       shutil.move(i,"output_files/ASSURE_VERILOG/"+i.split("/")[-1])

In [ ]:
# files=[([f"input_files/ASSURE_LOCKED/{i}/{k}" for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in k and "design" in k)][0],[f"input_files/ASSURE_LOCKED/{i}/{j}" for j in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in j and "oracle" in j)][0]) for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]
# files.sort()

In [ ]:
# import re
# import os
# files=[([f"input_files/ASSURE_LOCKED/{i}/{k}" for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in k and "design" in k)][0],[f"input_files/ASSURE_LOCKED/{i}/{j}" for j in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in j and "oracle" in j)][0]) for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]
# files.sort()
# notdone=[]
# for i in files[1:]:
#   design=i[0]
#   oracle=i[1]

#   tmp=open(design).read()
#   top_design = re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(tmp), re.DOTALL)[0][1]
#   tmp=open(oracle).read()
#   top_oracle = re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(tmp), re.DOTALL)[0][1]
#   # try:
#   #   print("DONE ",top_oracle)
#   obj=AST(file_path=oracle,rw="w",flag="v",top=top_oracle,filename=f"{top_oracle}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design
#   # except:
#   #   print("FAILED ",top_oracle)
#   #   notdone.append(oracle)
#   # try:
#   print("DONE ",top_design)
#   obj=AST(file_path=design,rw="w",flag="v",top=top_design,filename=f"{top_design}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False,locked=True)#Run to Read in Verilog Design
#   # except:
#   #   print("FAILED ",top_design)
#   #   notdone.append(design)
#   break

In [ ]:
# notdone=[]
# for i in os.listdir("input_files/benchmark_bench/original"):
#   top=i.split(".bench")[0]
#   unlocked_filename=os.path.join("input_files/benchmark_bench/original",i)
#   try:
#     obj=AST(file_path=unlocked_filename,rw="w",flag="b",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design
#   except:
#     notdone.append(unlocked_filename)
#   for j in os.listdir("input_files/benchmark_bench/rnd"):
#     if(i.split(".bench")[0] in j):
#       top=j.split(".bench")[0]
#       locked_filename=os.path.join("input_files/benchmark_bench/rnd",j)
#       print(j.split(".bench")[0],locked_filename)
#       try:
#         obj=AST(file_path=locked_filename,rw="w",flag="b",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False,locked=True)#Run to Read in Verilog Design
#       except:
#         notdone.append(locked_filename)